In [ ]:
import pandas as pd
import json
import datetime

In [ ]:
def format_date(date):

    formatted_date = '.'.join(str(date).split('-'))

    return formatted_date

In [ ]:
today = datetime.date.today()
today = format_date(today)

In [ ]:
from os import listdir

data_dir = "data"
files = listdir(data_dir)

latest_file = [f for f in files if f[-3:] == "csv"][-1]
latest_iter = latest_file[-5]

latest_df = pd.read_csv(data_dir + "\\" + latest_file)
latest_read_date = latest_df["date_readed"].max()


In [ ]:
raw_data_path = r"data\flights_data.json"
output_csv_path = f"data\\flights_{int(latest_iter) + 1}.csv"

In [ ]:
with open(raw_data_path) as f:
    data = json.load(f)["items"]

df = pd.DataFrame(data)
df["stops_num"] = df.stops.str["stops"]
max_stops = df["stops_num"].max()

df2 = pd.DataFrame(df.stops.str["stops_data"].tolist())

for i in range(max_stops):
    df[f"stop_{i+1}_duration"] = df2[i].str["duration"]
    df[f"stop_{i+1}_place"] = df2[i].str["place"]

df = df.drop('stops', axis=1)

In [ ]:
df = df[df["date_readed"] > latest_read_date]

In [ ]:
df[df["date_readed"] == today].sort_values("price").head(10)

In [ ]:
import os
from azure.storage.blob import BlobServiceClient

connect_str = os.getenv("AZURE_STORAGE_CONNECTION_STRING")
blob_service_client = BlobServiceClient.from_connection_string(connect_str)

In [ ]:
container_name = "mol"
container_client= blob_service_client.get_container_client(container_name)

In [ ]:
df.to_csv(output_csv_path)

blob_client = blob_service_client.get_blob_client(container=container_name, blob=output_csv_path)
print("\nUploading to Azure Storage as blob:\n\t" + output_csv_path)

with open(output_csv_path, "rb") as f:
    blob_client.upload_blob(f, overwrite=True)